# Project Objective

### Part 1 - Write python functions that solve a linear system using the following methods:
* Cholesky decomposition
* Guass-Seidal iteration
* Gradiet descent with exact line search
* Conjugate graidient method

### Part 2 - Test each method for $n=10$ and $n=100$. Measure and report CPU time for each method.
---

# Linear System

The linear system used in this project is as follows:

$$Ax=b, A \in \mathbb{R}^{n \times n}, A=A^T$$

where the matrix $A$ is defined as:
$$A=R^TR+nI, A \in \mathbb{R}^{n \times n}, A=A^T$$

where $R \in \mathbb{R}^{n \times n}$ is a random matrix with entries $R_{ij} \sim \text{Uniform}(0,1)$, $I$ is the indentity matrix, and $b \in \mathbb{R}$ is a random vector with entries $b_i \sim \text{Uniform}(0,1)$


---
# Method 1 - Cholesky Decomposion

Cholesky decomposition is a factorization of a positive definite matrix $A$ into the product of a lower triangular matrix $L$ and its transpose $L^T$. Mathematically, if $A$ is a positive definite matrix, then there exists a unique lower triangular matrix $L$ such that:

$$A = L L^T$$

Here's how the decomposition works step-by-step:

1. **Starting with Matrix $A$**:
   $$
   A =
   \begin{pmatrix}
   a_{11} & a_{12} & \cdots & a_{1n} \\
   a_{21} & a_{22} & \cdots & a_{2n} \\
   \vdots & \vdots & \ddots & \vdots \\
   a_{n1} & a_{n2} & \cdots & a_{nn}
   \end{pmatrix}
   $$

2. **Form Matrix $L$** such that:
   $$
   L =
   \begin{pmatrix}
   l_{11} & 0      & \cdots & 0      \\
   l_{21} & l_{22} & \cdots & 0      \\
   \vdots & \vdots & \ddots & \vdots \\
   l_{n1} & l_{n2} & \cdots & l_{nn}
   \end{pmatrix}
   $$

3. **Calculate Elements of $L$**:
   - **Diagonal Elements**:
     $$
     l_{ii} = \sqrt{a_{ii} - \sum_{k=1}^{i-1} l_{ik}^2}
     $$
   - **Off-diagonal Elements**:
     $$
     l_{ij} = \frac{1}{l_{jj}} \left(a_{ij} - \sum_{k=1}^{j-1} l_{ik} l_{jk}\right) \quad \text{for } i > j
     $$

For example, if $A$ is a 3x3 matrix:
$$
A =
\begin{pmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33}
\end{pmatrix}
$$

Then the elements of $L$ are calculated as follows:
$$
l_{11} = \sqrt{a_{11}}
$$
$$
l_{21} = \frac{a_{21}}{l_{11}}, \quad l_{22} = \sqrt{a_{22} - l_{21}^2}
$$
$$
l_{31} = \frac{a_{31}}{l_{11}}, \quad l_{32} = \frac{a_{32} - l_{31}l_{21}}{l_{22}}, \quad l_{33} = \sqrt{a_{33} - l_{31}^2 - l_{32}^2}
$$

Once all elements of $L$ are computed, we have a Cholesky decomposition $A = LL^T$.


In [48]:
def cholesky_decomposition(A):
    n = A.shape[0]
    L = np.zeros_like(A)

    for i in range(n):
        for j in range(i + 1):
            sum_k = sum(L[i][k] * L[j][k] for k in range(j))
            if i == j:  # Diagonal elements
                L[i][j] = np.sqrt(A[i][i] - sum_k)
            else:
                L[i][j] = (A[i][j] - sum_k) / L[j][j]

    return L

def solve_cholesky(A, b):
    L = cholesky_decomposition(A)

    # Forward substitution to solve Ly = b
    n = b.size
    y = np.zeros(n)
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]

    # Backward substitution to solve L^Tx = y
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.dot(L[i+1:, i], x[i+1:])) / L[i, i]

    return x


---
# Method 2 - Gauss Seidel

The Gauss-Seidel method is an iterative technique for solving a system of linear equations of the form $Ax = b$. It is particularly useful for large, sparse systems where direct methods may be impractical. The method iterates over each equation, using the most recent values of the solution vector $x$ to update the next values.

Mathematically, the Gauss-Seidel method can be described as follows:

Given a system of linear equations $Ax = b$, where $A$ is a matrix of coefficients, $x$ is the vector of unknowns, and $b$ is the vector of constants, the $i$-th equation can be written as:

$$ a_{ii} x_i^{(k+1)} = b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(k+1)} - \sum_{j=i+1}^{n} a_{ij} x_j^{(k)} $$

for $i = 1, 2, \ldots, n$, where $x_i^{(k+1)}$ is the updated value of $x_i$ at iteration $k+1$, and $x_i^{(k)}$ is the value of $x_i$ at iteration $k$.

The iterations continue until the solution converges, i.e., until the changes in the solution vector $x$ are below a specified tolerance.



In [49]:
def gauss_seidel_iteration(A, b, x0, tol=1e-10, max_iterations=1000):
    n = len(b)
    x = np.copy(x0)

    for k in range(max_iterations):
        x_new = np.copy(x)

        for i in range(n):
            sum1 = np.dot(A[i, :i], x_new[:i])
            sum2 = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (b[i] - sum1 - sum2) / A[i, i]

        # Check for convergence
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            return x_new, k + 1

        x = x_new

    return x, max_iterations

---
# Method 3 - Gradient Descent with Exact Line Search

Gradient Descent with exact line search is an iterative optimization algorithm used to minimize a function. For a linear system $Ax = b$, the objective is to minimize the quadratic function:

$$ f(x) = \frac{1}{2} x^T A x - b^T x $$

The algorithm updates the solution vector $x$ using the following steps:

1. **Compute the Gradient**: Calculate the gradient of the function at the current point $x_k$:
   $$ \nabla f(x_k) = A x_k - b $$

2. **Exact Line Search**: Determine the step size $\alpha_k$ by minimizing the function along the direction of the gradient:
   $$ \alpha_k = \frac{(A x_k - b)^T (A x_k - b)}{(A (A x_k - b))^T (A x_k - b)} $$

3. **Update**: Update the solution vector $x$ using the step size $\alpha_k$:
   $$ x_{k+1} = x_k - \alpha_k \nabla f(x_k) $$

The iterations continue until the gradient norm $\| \nabla f(x_k) \|$ is below a specified tolerance.


In [50]:
def gradient_descent(A, b, x0, max_iter=1000, tol=1e-10):
  x = x0
  grad = A @ x - b

  for k in range(max_iter):
    alpha = (grad.T @ grad) / (grad.T @ A @ grad) # Compute step size using exact line search
    alpha = alpha.item() # Convert to scalar

    x = x - alpha * grad # Update solution
    grad = A @ x - b # Compute new gradient

    if np.linalg.norm(grad) < tol: # Check convergence
      break

  return x, k + 1

---
# Method 4 - Conjugate Gradient Method

The Conjugate Gradient method is an iterative algorithm for solving systems of linear equations of the form $Ax = b$, where $A$ is a symmetric positive-definite matrix. The algorithm is particularly useful for large, sparse systems.

The method starts with an initial guess $x_0$ and iterates to find the solution by following these steps:

1. **Initialization**: Set $r_0 = b - Ax_0$ (residual) and $p_0 = r_0$ (search direction).

2. **Iterative Steps**:
   For $k = 0, 1, 2, \ldots$, do the following until convergence:
   \begin{align*}
   \alpha_k &= \frac{r_k^T r_k}{p_k^T A p_k} \\
   x_{k+1} &= x_k + \alpha_k p_k \\
   r_{k+1} &= r_k - \alpha_k A p_k \\
   \beta_k &= \frac{r_{k+1}^T r_{k+1}}{r_k^T r_k} \\
   p_{k+1} &= r_{k+1} + \beta_k p_k
   \end{align*}

3. **Convergence**: The iterations continue until the norm of the residual $\|r_k\|$ is below a specified tolerance.


In [51]:
def conjugate_gradient(A, b, x0, tol=1e-10, max_iter=1000):
  x = x0
  r = A @ x - b # Initial residual r0 = A @ x0 - b
  p = r.copy() # Initial direction p0 = r0

  for k in range(max_iter):
    Ap = A @ p
    alpha = (r.T @ r) / (p.T @ Ap) # Compute step size
    alpha = alpha.item() # Convert to scalar

    x = x - alpha * p # Update solution
    r_new = r - alpha * Ap # Update residual

    # Check for convergence
    if np.linalg.norm(r_new) < tol:
      break

    beta = (r_new.T @ r_new) / (r.T @ r) # Compute conjugate direction coefficient
    beta = beta.item() # Convert to scalar

    p = r_new + beta * p # Update direction
    r = r_new # Update residual

    return x, k + 1 # Return solution and number of iterations

---
# Part 2 - Testing Each Method

Testing the case where $n=10$

In [60]:
import time

n = 10
A = np.random.rand(n, n)
A = np.matmul(A.T, A) + n * np.eye(n) # Make A symmetric positive-definite
b = np.random.rand(n, 1)
x0 = np.zeros((n, 1))

#Cholesky
start_time = time.time()
x_cho = solve_cholesky(A, b)
time_cho = time.time() - start_time

#Gauss-seidel
start_time = time.time()
x_gs, iter_gs = gauss_seidel_iteration(A, b, x0)
time_gs = time.time()- start_time

# Gradient Descent
start_time = time.time()
x_gd, iter_gd = gradient_descent(A, b, x0)
time_gd = time.time()- start_time

# Conjugate Gradient
start_time = time.time()
x_cg, iter_cg = conjugate_gradient(A, b, x0)
time_cg = time.time()- start_time

print("\nCholesky:")
print(f"Time: {time_cho:.6f} seconds\n")
print("Gauss-Seidel:")
print(f"Iterations: {iter_gs}, Time: {time_gs:.6f} seconds\n")
print("Gradient Descent:")
print(f"Iterations: {iter_gd}, Time: {time_gd:.6f} seconds\n")
print("Conjugate Gradient:")
print(f"Iterations: {iter_cg}, Time: {time_cg:.6f} seconds\n")
print(np.linalg.norm(x_cho - x_gs))
print(np.linalg.norm(x_gs - x_gd))
print(np.linalg.norm(x_gd - x_cg))


Cholesky:
Time: 0.002450 seconds

Gauss-Seidel:
Iterations: 20, Time: 0.002704 seconds

Gradient Descent:
Iterations: 26, Time: 0.000754 seconds

Conjugate Gradient:
Iterations: 1, Time: 0.000163 seconds

0.28257446479407994
4.5532613205476103e-11
0.04078987506093969


<ipython-input-48-e3f2de999719>:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]


Testing the case where $n=100$

In [61]:
import numpy as np
import time

n = 100
A = np.random.rand(n, n)
A = np.matmul(A.T, A) + n * np.eye(n) # Make A symmetric positive-definite
b = np.random.rand(n, 1)
x0 = np.zeros((n, 1))

#Cholesky
start_time = time.time()
x_cho = solve_cholesky(A, b)
time_cho = time.time() - start_time

#Gauss-seidel
start_time = time.time()
x_gs, iter_gs = gauss_seidel_iteration(A, b, x0)
time_gs = time.time()- start_time

# Gradient Descent
start_time = time.time()
x_gd, iter_gd = gradient_descent(A, b, x0)
time_gd = time.time()- start_time

# Conjugate Gradient
start_time = time.time()
x_cg, iter_cg = conjugate_gradient(A, b, x0)
time_cg = time.time()- start_time

print("\nCholesky:")
print(f"Time: {time_cho:.6f} seconds\n")
print("Gauss-Seidel:")
print(f"Iterations: {iter_gs}, Time: {time_gs:.6f} seconds\n")
print("Gradient Descent:")
print(f"Iterations: {iter_gd}, Time: {time_gd:.6f} seconds\n")
print("Conjugate Gradient:")
print(f"Iterations: {iter_cg}, Time: {time_cg:.6f} seconds\n")
print(np.linalg.norm(x_cho - x_gs))
print(np.linalg.norm(x_gs - x_gd))
print(np.linalg.norm(x_gd - x_cg))

<ipython-input-48-e3f2de999719>:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]



Cholesky:
Time: 0.179223 seconds

Gauss-Seidel:
Iterations: 355, Time: 0.226717 seconds

Gradient Descent:
Iterations: 245, Time: 0.007392 seconds

Conjugate Gradient:
Iterations: 1, Time: 0.000222 seconds

0.38233768041986693
2.3340822296804838e-09
0.02561077462688726
